In [1]:
import numpy as np
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model


def getPrediction(filename):
    
    classes = ['bacterial_blight', 'Cassava Bacterial Blight (CBB)','Cassava Brown Streak Disease (CBSD)','Cassava Green Mottle (CGM)',
               'Cassava Mosaic Disease (CMD)',
               'curl_virus',
'fussarium_wilt',
'healthy',
'Healthy_cassava',
'healthy_cotton',
'Tomato___Bacterial_spot',
'Tomato___Early_blight',
'Tomato___healthy',
'Tomato___Late_blight',
'Tomato___Leaf_Mold',
'Tomato___Septoria_leaf_spot',
'Tomato___Spider_mites Two-spotted_spider_mite',
'Tomato___Target_Spot',
'Tomato___Tomato_mosaic_virus',
'Tomato___Tomato_Yellow_Leaf_Curl_Virus'
]
    le = LabelEncoder()
    le.fit(classes)
    le.inverse_transform([2])
    
    
    #Load model
    my_model=load_model("multidataset_model_vgg19.h5")
    
    SIZE = 128 #Resize to same size as training images
    img_path = 'static/images/'+filename
    img = np.asarray(Image.open(img_path).convert('RGB').resize((SIZE,SIZE)))
    
    img = img/255.      #Scale pixel values
    
    img = np.expand_dims(img, axis=0)  #Get it tready as input to the network       
    
    pred = my_model.predict(img) #Predict                    
    
    #Convert prediction to class name
    pred_class = le.inverse_transform([np.argmax(pred)])[0]
    print("Diagnosis is:", pred_class)
    return pred_class


#test_prediction = getPrediction('bact65.jpg')



In [ ]:
from flask import Flask, render_template, request, redirect, flash
from werkzeug.utils import secure_filename
import os

#Save images to the 'static' folder as Flask serves images from this directory
UPLOAD_FOLDER = 'static/images/'

#Create an app object using the Flask class. 
app = Flask(__name__, static_folder="static")

#Add reference fingerprint. 
#Cookies travel with a signature that they claim to be legit. 
#Legitimacy here means that the signature was issued by the owner of the cookie.
#Others cannot change this cookie as it needs the secret key. 
#It's used as the key to encrypt the session - which can be stored in a cookie.
#Cookies should be encrypted if they contain potentially sensitive information.
app.secret_key = "secret key"

#Define the upload folder to save images uploaded by the user. 
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

#Define the route to be home. 
#The decorator below links the relative route of the URL to the function it is decorating.
#Here, index function is with '/', our root directory. 
#Running the app sends us to index.html.
#Note that render_template means it looks for the file in the templates folder. 
@app.route('/')
def index():
    return render_template('index.html')

#Add Post method to the decorator to allow for form submission. 
@app.route('/', methods=['POST'])
def submit_file():
    if request.method == 'POST':
        if 'file' not in request.files:
            flash('No file part')
            return redirect(request.url)
        file = request.files['file']
        if file.filename == '':
            flash('No file selected for uploading')
            return redirect(request.url)
        if file:
            filename = secure_filename(file.filename)  #Use this werkzeug method to secure filename. 
            file.save(os.path.join(app.config['UPLOAD_FOLDER'],filename))
            #getPrediction(filename)
            label = getPrediction(filename)
            flash(label)
            full_filename = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            flash(full_filename)
            return redirect('/')


if __name__ == "__main__":
    app.run()
    app = Flask(__name__)
app.debug = True  # Enable debug mode


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [06/Nov/2024 14:55:38] "GET / HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


127.0.0.1 - - [06/Nov/2024 14:57:01] "POST / HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [06/Nov/2024 14:57:01] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [06/Nov/2024 14:57:01] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Nov/2024 14:57:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Nov/2024 14:57:01] "GET /static/images/0edb366d-1253-4498-a05a-8f666405edc6___Matt.S_CG_7862.JPG HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Nov/2024 14:57:01] "GET /static/images/0edb366d-1253-4498-a05a-8f666405edc6___Matt.S_CG_7862.JPG HTTP/1.1" 200 -


Diagnosis is: Tomato___Septoria_leaf_spot
